In [1]:
import cv2
import numpy as np
import os
from scipy import stats


video_data_path = 'HMDB51/video_data'
optical_flow_path = 'HMDB51/opt_flows'

In [29]:
def draw_flow_components(flow):
    horz = cv2.normalize(flow[...,0], None, 0, 255, cv2.NORM_MINMAX)     
    vert = cv2.normalize(flow[...,1], None, 0, 255, cv2.NORM_MINMAX)
    horz = horz.astype('uint8')
    vert = vert.astype('uint8')


    return horz, vert

In [34]:
def draw_hsv(flow, mask):
    

    # Computes the magnitude and angle of the 2D vectors 
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1]) 
      
    # Sets image hue according to the optical flow  
    # direction 
    mask[..., 0] = angle * 180 / np.pi / 2
      
    # Sets image value according to the optical flow 
    # magnitude (normalized) 
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX) 
      
    # Converts HSV to RGB (BGR) color representation 
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR) 
    return rgb

In [32]:
def save_optical_flow(video_path, output_dir):
    
    cap = cv2.VideoCapture(video_path)
    
    # Count total frames
    video_frames_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_step = max(video_frames_count // 16, 1)  
              
    # Reset video capture to first frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    ret, prev_frame = cap.read()
    
    if not ret:
      print(f"Video {video_path} can't be read")
      return  # Exit if the video can't be read
    
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_count = 0
    opt_flow_count = 0
    
    mask = np.zeros_like(prev_frame) 
    
    # Sets image saturation to maximum 
    mask[..., 1] = 255
    
    while True:
        ret, next_frame = cap.read()
        if not ret:
            break  # Exit loop if no more frames are available

        if frame_count % frame_step == 0:
            next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
            
            '''
            - pyr_scale (0.5): Scaling factor for constructing the image pyramid. 
              0.5 indicates that each layer of the pyramid is half the size of the previous layer.
            - levels (3): The number of pyramid layers.
            - winsize (15): The averaging window size. The size of the window used to smooth the optical flow. 
            - iterations (3): The number of iterations the algorithm will use at each pyramid level.
            - poly_n (5): The size of the pixel neighborhood used to find polynomial expansion in each pixel.  
            - poly_sigma (1.2): Standard deviation of the Gaussian that is used to smooth derivatives for the polynomial expansion.
            - flags (0): Commonly used value is 0.
            '''
            flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                                    
            ### CASE OF VERTICAL AND HORIZONTAL ###
            # flow_filename_horizontal = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(video_path))[0]}_opt_flow_{opt_flow_count}_hori.jpeg")
            # flow_filename_vertical = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(video_path))[0]}_opt_flow_{opt_flow_count}_vert.jpeg")
            
            # flow_image_horizontal, flow_image_vertical = draw_flow_components(flow)
                        
            # cv2.imwrite(flow_filename_horizontal, flow_image_horizontal)
            # cv2.imwrite(flow_filename_vertical, flow_image_vertical)
            
            ### CASE OF COLLORFUL ###
            flow_filename = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(video_path))[0]}_opt_flow_{opt_flow_count}.jpeg")
            flow_image = draw_hsv(flow, mask)
            cv2.imwrite(flow_filename, flow_image)
            
            opt_flow_count += 1
    
        frame_count += 1
        
    return opt_flow_count

In [35]:
files_per_action = []
min_opt_frames = 25

for action_class in os.listdir(video_data_path):
    counter = 0
    action_class_path = os.path.join(video_data_path, action_class)
    output_class_path = os.path.join(optical_flow_path, action_class)

    if not os.path.exists(output_class_path):
        os.makedirs(output_class_path)

    for video in os.listdir(action_class_path):
        counter += 1
        video_path = os.path.join(action_class_path, video)
        
        opt_frames_count = save_optical_flow(video_path, output_class_path)
        
        if opt_frames_count < min_opt_frames:
            min_opt_frames = opt_frames_count
        
    files_per_action.append(counter)
    
    
print(files_per_action)
print("Min number of frames:", min_opt_frames)

KeyboardInterrupt: 

In [128]:
import re

def process_action_class(action_class_path):
    # Dictionary to hold frame count for each video
    video_frame_counts = {}

    # Identify and count frames for each video
    for file_name in os.listdir(action_class_path):
        match = re.match(r"(.+)_opt_flow_(\d+)\.jpeg", file_name)
        if match:
            video_name, frame_num = match.groups()
            frame_num = int(frame_num)

            if video_name not in video_frame_counts:
                video_frame_counts[video_name] = []

            video_frame_counts[video_name].append((frame_num, file_name))

    # Remove excess frames
    for video_name, frames in video_frame_counts.items():
        if len(frames) > min_opt_frames:
            # Sort frames by frame number and keep the first 'min_opt_frames'
            frames_to_remove = sorted(frames, key=lambda x: x[0])[min_opt_frames:]
            for _, frame_name in frames_to_remove:
                frame_path = os.path.join(action_class_path, frame_name)
                os.remove(frame_path)  # Delete the frame



for action_class in os.listdir(optical_flow_path):
    class_path = os.path.join(optical_flow_path, action_class)
    if os.path.isdir(class_path):
        process_action_class(class_path)
